<a href="https://colab.research.google.com/github/mgnarag/painting_restoration/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!ls drive/My\ Drive
file_path = "/content/drive/MyDrive/Baumgartner screenshots/"

'107.1 syllabus 1SAY1920 FINAL.gdoc'
'1C-01 Narag et al (1).pdf'
'1C-01 Narag et al.pdf'
'1FA971504AC646559D05A8ED6341AFB3[1025340].png'
'1H-04 Narag et al.pdf'
 20220802_182749.heic
 20220802_182831.heic
 20220802_183259.heic
 20220803_125819.heic
 20220803_125842.heic
 20220803_144229.heic
 20220803_144422.heic
 20220803_160724.heic
 20220803_160751.heic
 20220803_165240.heic
 20220803_165318.heic
 20220803_174701.heic
 20220803_174706.heic
 20220804_175217.heic
 20220804_175443.heic
 20220804_175616.heic
 20220804_181917.heic
 20220804_182007.heic
 20220804_182117.heic
 2S22-23_LE3-Regular-Set-A-FINAL-KEY.pdf
'3 classes (polarized)CM_alexnet.png'
'3 classes (polarized)CM_vgg.png'
'A4_Physics 71 TWHFU-3_gradesheet (1).pdf'
'A4_Physics 71 TWHFU-3_officialgrades (1).pdf'
'A4_Physics 71 TWHFU-3_officialgrades.gdoc'
'Abstract - NARAG, Mark Jeremy.pdf'
'Annex 1 (copy of Journal).pdf'
'Annex 2 (Proof of employments).pdf'
'App Physics 181 THY-FX-2_studentcontactlist.xlsx'
 Architectural_des

## Installing packages

In [ ]:
import time
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from PIL import Image, ImageOps
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from scipy.ndimage import zoom

import cv2
from skimage import color

from tensorflow import keras
from PIL import Image, ImageOps
import numpy as np
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 236958876
from sklearn.model_selection import train_test_split

## Defining functions

In [ ]:
def zoom_and_resize(img, zoom_factor):
    height, width = img.shape[:2]
    crop_top = int((height - height / zoom_factor) / 2)# Calculate the region to crop around the center
    crop_bottom = int(height - crop_top)
    crop_left = int((width - width / zoom_factor) / 2)
    crop_right = int(width - crop_left)
    cropped_image = img[crop_top:crop_bottom, crop_left:crop_right]    # Crop the image
    resized_image = cv2.resize(cropped_image, (width, height))# Resize the zoomed image back to the original dimensions
    return resized_image

def crop(im):
    width, height = im.size
    data = []
    step = int(size * 0.80)  # Adjust the step size for cropping
    for j in range(0, height, step):
        for i in range(0, width, step):
            if i + size <= width and j + size <= height:
                im1 = im.crop((i, j, i + size, j + size))
                im1 = np.array(im1).astype(np.float32)
                data.append(im1/255)
                im1 = np.rot90(im1)
                data.append(im1/255)
                #im1 = np.rot90(im1)
                #data.append(im1/255)
                #im1 = np.rot90(im1)
                #data.append(im1/255)
                #for z in zoom_factor:
                #    zoomed_img = zoom_and_resize(im1, z)
                #    data.append(zoomed_img/255)
    return data

def rgb_lab_normalized(im):
    rgb_array = np.array(im) # Convert to numpy array
    rgb_array_normalized = rgb_array / 255.0 # Normalize RGB values from 0 to 1
    lab_array_normalized = color.rgb2lab(rgb_array_normalized) # Convert RGB to LAB colorspace
    lab_array_normalized[..., 0] = (lab_array_normalized[..., 0]) / 100.0 # Scale LAB values to range [0, 1]
    lab_array_normalized[..., 1] = (lab_array_normalized[..., 1] + 128) / 255.0
    lab_array_normalized[..., 2] = (lab_array_normalized[..., 2] + 128) / 255.0
    lab_image_normalized = Image.fromarray((lab_array_normalized * 255).astype(np.uint8), mode='LAB')# Convert LAB array back to image
    return lab_image_normalized

## Testing the model

In [ ]:
filename = str("INPUT_[05] Comprimise-d.png")

### **Model RGB:**

In [ ]:
n_size = 32
size = 32

def rgb_L_ab(rgb_image, test_L):
    lab_array = color.rgb2lab(np.array(rgb_image))# Convert RGB to LAB colorspace
    lab_array[..., 0] = test_L #Change L channel to input
    rgb_array = color.lab2rgb(lab_array)# Convert back LAB to RGB colorspace
    rgb_array = (rgb_array * 255).astype(np.uint8)# Scale RGB values back to the range [0, 255]
    rgb_array = Image.fromarray(rgb_array, mode='RGB')# Convert RGB array back to image
    return rgb_array

def crop_(im):
    width, height = im.size
    data = []
    for j in range(0,int(height/n_size)):
        for i in range(0,int(width/n_size)):
            im1 = im.crop((0 + (n_size*i), 0 + (n_size*j), n_size + (n_size*i), n_size + (n_size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            im1 = im1/255
            data.append(im1)
    return data

#----------------READING THE TEST IMAGE--------------------#
filename = str("INPUT_[09] Preserve and Protect.png")
test = Image.open(file_path+ "Testing/" + filename).convert('RGB')
test_L = color.rgb2lab(np.array(test))
test_L = test_L[..., 0]
w_dirty, h_dirty = test.size
#----------------------------------------------------------#

autoencoder = keras.models.load_model(file_path + 'Model/unet_'+str(n_size)+'_rgb')

xx = int(w_dirty/n_size)
final=[]

for portion in range(0,xx):
    im1 = test.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
    w1, h1 = im1.size
    w = int(w1/n_size)
    h = int(h1/n_size)
    neverbeforeseen = np.array(crop_(im1))
    decoded_imgs = autoencoder.predict(neverbeforeseen)
    col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
    for i in range(2,h):
        col = np.vstack((col,decoded_imgs[i]))
    y = col
    y = (y * 255).astype('uint8')
    if portion == 0:
        final = y
    if portion > 0:
        final = np.hstack((final,y))

#----------------TEST ON FULL RGB--------------------#
final_rgb = final
final_rgb = np.squeeze(final_rgb)
reconstructed_rgb = Image.fromarray(final_rgb)
reconstructed_rgb.save(file_path + "Testing/"+"Model_1_"+filename)
plt.figure(), plt.imshow(reconstructed_rgb)

#----------------TEST ON L from input and AB from convert RGB2LAB of model--------------------#
final_lab = rgb_L_ab(final, test_L)
final_lab = np.squeeze(final_lab)
reconstructed_lab = Image.fromarray(final_lab)
reconstructed_lab.save(file_path + "Testing/"+"Model_2_"+filename)
plt.figure(), plt.imshow(reconstructed_lab)

### **Model LAB:**

In [ ]:
from tensorflow import keras
from PIL import Image, ImageOps
import numpy as np
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 236958876

n_size = 32
size = n_size

def lab_rgb_unnormalized(lab_image):
    lab_array = np.array(lab_image/255)# Convert LAB image to numpy array
    lab_array[..., 0] = lab_array[..., 0] * 100.0# Scale LAB values back to their original ranges
    lab_array[..., 1] = (lab_array[..., 1] * 255.0) - 128
    lab_array[..., 2] = (lab_array[..., 2] * 255.0) - 128
    rgb_array_unnormalized = color.lab2rgb(lab_array)# Convert LAB to RGB colorspace
    rgb_array_unnormalized = (rgb_array_unnormalized * 255).astype(np.uint8)# Scale RGB values back to the range [0, 255]
    rgb_image_unnormalized = Image.fromarray(rgb_array_unnormalized, mode='RGB')# Convert RGB array back to image
    return rgb_image_unnormalized

def AB_rgb_unnormalized(lab_image, test_L):
    lab_array = np.array(lab_image/255)# Convert LAB image to numpy array
    lab_array[..., 0] = test_L #replacing L from the original input
    lab_array[..., 1] = (lab_array[..., 1] * 255.0) - 128
    lab_array[..., 2] = (lab_array[..., 2] * 255.0) - 128
    rgb_array_unnormalized = color.lab2rgb(lab_array)# Convert LAB to RGB colorspace
    rgb_array_unnormalized = (rgb_array_unnormalized * 255).astype(np.uint8)# Scale RGB values back to the range [0, 255]
    rgb_image_unnormalized = Image.fromarray(rgb_array_unnormalized, mode='RGB')# Convert RGB array back to image
    return rgb_image_unnormalized

def crop_(im):
    width, height = im.size
    data = []
    for j in range(0,int(height/n_size)):
        for i in range(0,int(width/n_size)):
            im1 = im.crop((0 + (n_size*i), 0 + (n_size*j), n_size + (n_size*i), n_size + (n_size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            im1 = im1/255
            data.append(im1)
    return data

#----------------READING THE TEST IMAGE--------------------#
test = Image.open(file_path+ "Testing/" + filename).convert('RGB')
test_L = color.rgb2lab(np.array(test))
test_L = test_L[..., 0]
test = rgb_lab_normalized(test)
w_dirty, h_dirty = test.size
#----------------------------------------------------------#

autoencoder = keras.models.load_model(file_path + 'Model/unet_'+str(n_size)+'_lab')

xx = int(w_dirty/n_size)
final=[]
for portion in range(0,xx):
    im1 = test.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
    w1, h1 = im1.size
    w = int(w1/n_size)
    h = int(h1/n_size)
    neverbeforeseen = np.array(crop_(im1))
    decoded_imgs = autoencoder.predict(neverbeforeseen)
    col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
    for i in range(2,h):
        col = np.vstack((col,decoded_imgs[i]))
    y = col
    y = (y * 255).astype('uint8')
    if portion == 0:
        final = y
    if portion > 0:
        final = np.hstack((final,y))

#----------------TEST ON FULL LAB--------------------#
final_LAB = lab_rgb_unnormalized(final)
final_LAB = np.squeeze(final_LAB)
reconstructed_LAB = Image.fromarray(final_LAB)
reconstructed_LAB.save(file_path + "Testing/"+"Method_3_"+filename)
plt.figure(), plt.imshow(reconstructed_LAB)

#----------------TEST ON L from input and AB from model--------------------#
final_L_AB = AB_rgb_unnormalized(final, test_L)
final_L_AB = np.squeeze(final_L_AB)
reconstructed_L_AB = Image.fromarray(final_L_AB)
reconstructed_L_AB.save(file_path + "Testing/"+"Method_4_"+filename)
plt.figure(), plt.imshow(reconstructed_L_AB)

### **Model AB**

In [ ]:
from tensorflow import keras
from PIL import Image, ImageOps
import numpy as np
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 236958876

n_size = 32
size = n_size

def rgb_lab_normalized(im):
    rgb_array = np.array(im) # Convert to numpy array
    rgb_array_normalized = rgb_array / 255.0 # Normalize RGB values from 0 to 1
    lab_array_normalized = color.rgb2lab(rgb_array_normalized) # Convert RGB to LAB colorspace
    lab_array_normalized[..., 0] = (lab_array_normalized[..., 0]) / 100.0 # Scale LAB values to range [0, 1]
    lab_array_normalized[..., 1] = (lab_array_normalized[..., 1] + 128) / 255.0
    lab_array_normalized[..., 2] = (lab_array_normalized[..., 2] + 128) / 255.0
    lab_image_normalized = Image.fromarray((lab_array_normalized * 255).astype(np.uint8), mode='LAB')# Convert LAB array back to image
    return lab_image_normalized

def AB_rgb_unnormalized(ab_image, test_L):
    ab_array = np.array(ab_image)/255# Convert LAB image to numpy array
    ab_array[..., 0] = (ab_array[..., 0] * 255.0) - 128
    ab_array[..., 1] = (ab_array[..., 1] * 255.0) - 128
    LAB_array = np.dstack((test_L, ab_array[:,:,0], ab_array[:,:,1]))
    rgb_array_unnormalized = color.lab2rgb(LAB_array)# Convert LAB to RGB colorspace
    rgb_array_unnormalized = (rgb_array_unnormalized * 255).astype(np.uint8)# Scale RGB values back to the range [0, 255]
    rgb_image_unnormalized = Image.fromarray(rgb_array_unnormalized, mode='RGB')# Convert RGB array back to image
    return rgb_image_unnormalized

def crop_(im):
    width, height = im.size
    data = []
    for j in range(0,int(height/n_size)):
        for i in range(0,int(width/n_size)):
            im1 = im.crop((0 + (n_size*i), 0 + (n_size*j), n_size + (n_size*i), n_size + (n_size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            data.append(im1)
    return data



#----------------READING THE TEST IMAGE--------------------#
test = Image.open(file_path+ "Testing/" + filename).convert('RGB')
test_L = color.rgb2lab(np.array(test))
test_L = np.array(test_L[..., 0]).astype('uint8')
test = rgb_lab_normalized(test)
#test = test[:,:,1:3]
#test = np.array(test).astype(np.float32)/255.0
w_dirty, h_dirty = test.size
#----------------------------------------------------------#

autoencoder = keras.models.load_model(file_path + 'Model/unet_'+str(n_size)+'_ab')

xx = int(w_dirty/n_size)
final=[]
for portion in range(0,xx):
    im1 = test.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
    w1, h1 = im1.size
    w = int(w1/n_size)
    h = int(h1/n_size)
    neverbeforeseen = np.array(crop_(im1))
    neverbeforeseen = neverbeforeseen[:,:,:,1:3]
    neverbeforeseen = np.array(neverbeforeseen).astype(np.float32)/255.0
    decoded_imgs = autoencoder.predict(neverbeforeseen)
    col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
    for i in range(2,h):
        col = np.vstack((col,decoded_imgs[i]))
    y = col
    y = (y * 255).astype('uint8')
    if portion == 0:
        final = y
    if portion > 0:
        final = np.hstack((final,y))

#----------------TEST ON L from input and AB from model--------------------#
final_L_AB = AB_rgb_unnormalized(final, test_L)
final_L_AB = np.squeeze(final_L_AB)
reconstructed_L_AB = Image.fromarray(final_L_AB)
reconstructed_L_AB.save(file_path + "Testing/"+"Model_5_"+filename)
plt.figure(), plt.imshow(reconstructed_L_AB)

### **Model AB plus Model L**

In [ ]:
from tensorflow import keras
from PIL import Image, ImageOps
import numpy as np
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 236958876

n_size = 32
size = 32

def rgb_lab_normalized(im):
    rgb_array = np.array(im) # Convert to numpy array
    rgb_array_normalized = rgb_array / 255.0 # Normalize RGB values from 0 to 1
    lab_array_normalized = color.rgb2lab(rgb_array_normalized) # Convert RGB to LAB colorspace
    lab_array_normalized[..., 0] = (lab_array_normalized[..., 0]) / 100.0 # Scale LAB values to range [0, 1]
    lab_array_normalized[..., 1] = (lab_array_normalized[..., 1] + 128) / 255.0
    lab_array_normalized[..., 2] = (lab_array_normalized[..., 2] + 128) / 255.0
    lab_image_normalized = Image.fromarray((lab_array_normalized * 255).astype(np.uint8), mode='LAB')# Convert LAB array back to image
    return lab_image_normalized

def L_AB_rgb_unnormalized(ab_image, final_L):
    ab_image = ab_image/255.0
    final_L = final_L/255.0
    final_L[..., 0] = final_L[..., 0]*100.0
    ab_image[..., 0] = (ab_image[..., 0] * 255.0) - 128
    ab_image[..., 1] = (ab_image[..., 1] * 255.0) - 128
    LAB_array = np.dstack((final_L, ab_image[:,:,0], ab_image[:,:,1]))
    rgb_array_unnormalized = color.lab2rgb(LAB_array)# Convert LAB to RGB colorspace
    rgb_array_unnormalized = (rgb_array_unnormalized * 255).astype(np.uint8)# Scale RGB values back to the range [0, 255]
    rgb_image_unnormalized = Image.fromarray(rgb_array_unnormalized, mode='RGB')# Convert RGB array back to image
    return rgb_image_unnormalized

def crop_(im):
    width, height = im.size
    data = []
    for j in range(0,int(height/n_size)):
        for i in range(0,int(width/n_size)):
            im1 = im.crop((0 + (n_size*i), 0 + (n_size*j), n_size + (n_size*i), n_size + (n_size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            data.append(im1)
    return data


#----------------READING THE TEST IMAGE--------------------#
test = Image.open(file_path+ "Testing/" + filename).convert('RGB')
test = rgb_lab_normalized(test)
w_dirty, h_dirty = test.size
#----------------------------------------------------------#

#----------------GETTING AB--------------------#
autoencoder = keras.models.load_model(file_path + 'Model/unet_'+str(n_size)+'_ab')

xx = int(w_dirty/n_size)
final=[]
for portion in range(0,xx):
    im1 = test.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
    w1, h1 = im1.size
    w = int(w1/n_size)
    h = int(h1/n_size)
    neverbeforeseen = np.array(crop_(im1))
    neverbeforeseen = neverbeforeseen[:,:,:,1:3]
    neverbeforeseen = np.array(neverbeforeseen).astype(np.float32)/255.0
    decoded_imgs = autoencoder.predict(neverbeforeseen)
    col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
    for i in range(2,h):
        col = np.vstack((col,decoded_imgs[i]))
    y = col
    y = (y * 255).astype('uint8')
    if portion == 0:
        final = y
    if portion > 0:
        final = np.hstack((final,y))

#----------------GETTING L--------------------#
autoencoder = keras.models.load_model(file_path + 'Model/unet_'+str(n_size)+'_L')

final_L=[]
for portion in range(0,xx):
    im1 = test.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
    w1, h1 = im1.size
    w = int(w1/n_size)
    h = int(h1/n_size)
    neverbeforeseen = np.array(crop_(im1))
    neverbeforeseen = neverbeforeseen[:,:,:,0:1]
    neverbeforeseen = np.array(neverbeforeseen).astype(np.float32)/255.0
    decoded_imgs = autoencoder.predict(neverbeforeseen)
    col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
    for i in range(2,h):
        col = np.vstack((col,decoded_imgs[i]))
    y = col
    y = (y * 255).astype('uint8')
    if portion == 0:
        final_L = y
    if portion > 0:
        final_L = np.hstack((final_L,y))

#----------------TEST ON L from model and AB from model--------------------#
final_L_AB = L_AB_rgb_unnormalized(final, final_L)
final_L_AB = np.squeeze(final_L_AB)
reconstructed_L_AB = Image.fromarray(final_L_AB)
reconstructed_L_AB.save(file_path + "Testing/"+"Method_6_"+filename)
plt.figure(), plt.imshow(reconstructed_L_AB)

## COMPUTE SSIM SCORES

In [ ]:
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim

# Function to open image and convert to numpy array
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB mode
    return np.array(image)

image_path1 = file_path + 'Testing/OUTPUT_[05] Comprimise-d.png'
image_path2 = file_path + "Testing/"+"Method_3_"+str(size)+"LAB_"+filename

# Load images
image1 = load_image(image_path1)
image2 = load_image(image_path2)

# Ensure the images have the same dimensions
if image1.shape != image2.shape:
    raise ValueError("Input images must have the same dimensions.")

# Compute SSIM for each channel and average the results
ssim_index_r = ssim(image1[:, :, 0], image2[:, :, 0], data_range=image1[:, :, 0].max() - image1[:, :, 0].min())
ssim_index_g = ssim(image1[:, :, 1], image2[:, :, 1], data_range=image1[:, :, 1].max() - image1[:, :, 1].min())
ssim_index_b = ssim(image1[:, :, 2], image2[:, :, 2], data_range=image1[:, :, 2].max() - image1[:, :, 2].min())

ssim_index_rgb = (ssim_index_r + ssim_index_g + ssim_index_b) / 3

print(f"SSIM for R channel: {ssim_index_r}")
print(f"SSIM for G channel: {ssim_index_g}")
print(f"SSIM for B channel: {ssim_index_b}")
print(f"Average SSIM for RGB image(Method3): {ssim_index_rgb}")


SSIM for R channel: 0.509358346147561
SSIM for G channel: 0.5081811788128479
SSIM for B channel: 0.4024215496836909
Average SSIM for RGB image(Method3): 0.4733203582146999


In [ ]:
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim

# Function to open image and convert to numpy array
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')  # Ensure image is in RGB mode
    return np.array(image)

image_path1 = file_path + 'Testing/OUTPUT_[05] Comprimise-d.png'
image_path2 = file_path + "Testing/"+"Method_4_"+str(size)+"L_AB_"+filename

# Load images
image1 = load_image(image_path1)
image2 = load_image(image_path2)

# Ensure the images have the same dimensions
if image1.shape != image2.shape:
    raise ValueError("Input images must have the same dimensions.")

# Compute SSIM for each channel and average the results
ssim_index_r = ssim(image1[:, :, 0], image2[:, :, 0], data_range=image1[:, :, 0].max() - image1[:, :, 0].min())
ssim_index_g = ssim(image1[:, :, 1], image2[:, :, 1], data_range=image1[:, :, 1].max() - image1[:, :, 1].min())
ssim_index_b = ssim(image1[:, :, 2], image2[:, :, 2], data_range=image1[:, :, 2].max() - image1[:, :, 2].min())

ssim_index_rgb = (ssim_index_r + ssim_index_g + ssim_index_b) / 3

print(f"SSIM for R channel: {ssim_index_r}")
print(f"SSIM for G channel: {ssim_index_g}")
print(f"SSIM for B channel: {ssim_index_b}")
print(f"Average SSIM for RGB image(Method4): {ssim_index_rgb}")


SSIM for R channel: 0.5487580309946481
SSIM for G channel: 0.5692294383749303
SSIM for B channel: 0.4253035732161732
Average SSIM for RGB image(Method4): 0.514430347528584


In [ ]:
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim

def compute_average_ssim(image1_path, image2_path):
    # Open the images using Pillow
    image1 = Image.open(image1_path).convert('RGB')
    image2 = Image.open(image2_path).convert('RGB')

    # Convert images from RGB to LAB color space using Pillow
    lab_image1 = image1.convert('LAB')
    lab_image2 = image2.convert('LAB')

    # Convert to numpy arrays
    lab_image1 = np.array(lab_image1)
    lab_image2 = np.array(lab_image2)

    # Extract the A and B channels
    a_channel1, b_channel1 = lab_image1[:, :, 1], lab_image1[:, :, 2]
    a_channel2, b_channel2 = lab_image2[:, :, 1], lab_image2[:, :, 2]

    # Compute SSIM for the A channel
    ssim_a = ssim(a_channel1, a_channel2)

    # Compute SSIM for the B channel
    ssim_b = ssim(b_channel1, b_channel2)

    # Average the SSIM values
    average_ssim = (ssim_a + ssim_b) / 2

    return average_ssim

# Paths to the images
image1_path = file_path + 'Testing/OUTPUT_[05] Comprimise-d.png'
image2_path = image_path2 = file_path + "Testing/"+"Method_3_"+str(size)+"LAB_"+filename

# Compute the average SSIM
average_ssim_value = compute_average_ssim(image1_path, image2_path)
print(f"Average SSIM (A and B channels (Method3)): {average_ssim_value}")


Average SSIM (A and B channels (Method3)): 0.4829091381886401


In [ ]:
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim

def compute_average_ssim(image1_path, image2_path):
    # Open the images using Pillow
    image1 = Image.open(image1_path).convert('RGB')
    image2 = Image.open(image2_path).convert('RGB')

    # Convert images from RGB to LAB color space using Pillow
    lab_image1 = image1.convert('LAB')
    lab_image2 = image2.convert('LAB')

    # Convert to numpy arrays
    lab_image1 = np.array(lab_image1)
    lab_image2 = np.array(lab_image2)

    # Extract the A and B channels
    a_channel1, b_channel1 = lab_image1[:, :, 1], lab_image1[:, :, 2]
    a_channel2, b_channel2 = lab_image2[:, :, 1], lab_image2[:, :, 2]

    # Compute SSIM for the A channel
    ssim_a = ssim(a_channel1, a_channel2)

    # Compute SSIM for the B channel
    ssim_b = ssim(b_channel1, b_channel2)

    # Average the SSIM values
    average_ssim = (ssim_a + ssim_b) / 2

    return average_ssim

# Paths to the images
image1_path = file_path + 'Testing/OUTPUT_[05] Comprimise-d.png'
image2_path = image_path2 = file_path + "Testing/"+"Method_4_"+str(size)+"L_AB_"+filename

# Compute the average SSIM
average_ssim_value = compute_average_ssim(image1_path, image2_path)
print(f"Average SSIM (A and B channels (Method4)): {average_ssim_value}")


Average SSIM (A and B channels (Method4)): 0.4836187382989704
